# Import Libraries

In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import os
import pandas as pd

# Obtain Data

Create BigQuery client

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= 'world-bank-indicators-3ccb8c5c2658.json'
client = bigquery.Client()

Get census bureau international dataset

In [3]:
dataset_id = client.dataset("census_bureau_international", project="bigquery-public-data")
dataset = client.get_dataset(dataset_id)

View census bureau international dataset tables

In [4]:
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)

age_specific_fertility_rates
birth_death_growth_rates
country_names_area
midyear_population
midyear_population_5yr_age_sex
midyear_population_age_sex
midyear_population_agespecific
mortality_life_expectancy


Get fertility rates table

In [5]:
table_id = dataset_id.table("age_specific_fertility_rates")
table = client.get_table(table_id)
[print(item) for item in table.schema]
client.list_rows(table, max_results=5).to_dataframe()

SchemaField('country_code', 'STRING', 'REQUIRED', 'Federal Information Processing Standard (FIPS) country/area code', (), None)
SchemaField('country_name', 'STRING', 'NULLABLE', 'Country or area name', (), None)
SchemaField('year', 'INTEGER', 'REQUIRED', 'Year', (), None)
SchemaField('fertility_rate_15_19', 'FLOAT', 'NULLABLE', 'Age specific fertility rate for age 15-19 (births per 1,000 population)', (), None)
SchemaField('fertility_rate_20_24', 'FLOAT', 'NULLABLE', 'Age specific fertility rate for age 20-24 (births per 1,000 population)', (), None)
SchemaField('fertility_rate_25_29', 'FLOAT', 'NULLABLE', 'Age specific fertility rate for age 25-29 (births per 1,000 population)', (), None)
SchemaField('fertility_rate_30_34', 'FLOAT', 'NULLABLE', 'Age specific fertility rate for age 30-34 (births per 1,000 population)', (), None)
SchemaField('fertility_rate_35_39', 'FLOAT', 'NULLABLE', 'Age specific fertility rate for age 35-39 (births per 1,000 population)', (), None)
SchemaField('fert

/Users/blantj/opt/anaconda3/lib/python3.7/site-packages/google/cloud/bigquery/table.py:1690: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  if not self._validate_bqstorage(bqstorage_client, create_bqstorage_client):


,country_code,country_name,year,fertility_rate_15_19,fertility_rate_20_24,fertility_rate_25_29,fertility_rate_30_34,fertility_rate_35_39,fertility_rate_40_44,fertility_rate_45_49,total_fertility_rate,gross_reproduction_rate,sex_ratio_at_birth
0,AA,Aruba,1981,39.0,118.0,118.0,75.0,30.0,6.0,1.0,1.9350,0.9762,0.9822
1,AA,Aruba,1982,41.8,119.9,119.3,76.1,30.7,6.0,0.9,1.9737,0.9963,0.9811
2,AA,Aruba,1983,44.6,121.9,120.6,77.2,31.4,6.0,0.8,2.0124,1.0155,0.9817
3,AA,Aruba,1984,47.5,123.9,121.8,78.2,32.1,6.0,0.7,2.0511,1.0348,0.9822
4,AA,Aruba,1985,50.3,125.8,123.1,79.3,32.8,6.0,0.6,2.0898,1.0552,0.9804


Query fertility rates data from bigquery

In [6]:
fertility_query = """
        SELECT country_code, country_name, year, total_fertility_rate
        FROM `bigquery-public-data.census_bureau_international.age_specific_fertility_rates`
        WHERE year < 2019"""
fertility_query_job = client.query(fertility_query)
fertility_df = fertility_query_job.to_dataframe()
fertility_df.to_csv('Excel/fertility.csv', index=False)

Read fertility rate data into pandas

In [7]:
fertility_df = pd.read_csv('Excel/fertility.csv')
print(fertility_df.shape)
fertility_df.head()

(7720, 4)


,country_code,country_name,year,total_fertility_rate
0,AA,Aruba,1981,1.9350
1,AA,Aruba,1982,1.9737
2,AA,Aruba,1983,2.0124
3,AA,Aruba,1984,2.0511
4,AA,Aruba,1985,2.0898


Filter fertility rates data to only include country fertility values for the most recent year

In [8]:
def fertility_filter(row):
    years = fertility_df[fertility_df['country_code']==row['country_code']]['year'].values
    if row['year'] == max(years):
        return 1
    else:
        return 0

most_recent_years = fertility_df.apply(fertility_filter, axis=1)
fertility_df['filter'] = most_recent_years
fertility_df_filtered = fertility_df[fertility_df['filter']==1]
fertility_df_filtered.drop(columns='filter', inplace=True)
print(fertility_df_filtered.shape)
fertility_df_filtered.head(5)

(228, 4)


/Users/blantj/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,country_code,country_name,year,total_fertility_rate
37,AA,Aruba,2018,1.8324
65,AC,Antigua and Barbuda,2018,1.9906
89,AE,United Arab Emirates,2018,1.7269
129,AF,Afghanistan,2018,5.0200
161,AG,Algeria,2018,2.6646


Create fertility df column for replacement fertility class

In [9]:
def fertility_class(row):
    if row['total_fertility_rate'] >= 2:
        return 1
    else:
        return 0
fertility_df_filtered['replacement'] = fertility_df_filtered.apply(fertility_class, axis=1)
fertility_df_filtered.head()

/Users/blantj/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,country_code,country_name,year,total_fertility_rate,replacement
37,AA,Aruba,2018,1.8324,0
65,AC,Antigua and Barbuda,2018,1.9906,0
89,AE,United Arab Emirates,2018,1.7269,0
129,AF,Afghanistan,2018,5.0200,1
161,AG,Algeria,2018,2.6646,1


Get world bank wdi dataset

In [10]:
wb_dataset_id = client.dataset("world_bank_wdi", project="bigquery-public-data")
wb_dataset = client.get_dataset(wb_dataset_id)

View tables in world bank dataset

In [11]:
wb_tables = list(client.list_tables(wb_dataset))
for table in wb_tables:
    print(table.table_id)

country_series_definitions
country_summary
footnotes
indicators_data
series_summary
series_time


View world bank indicators_data table

In [12]:
indicator_table_id = wb_dataset_id.table("indicators_data")
indicator_table = client.get_table(indicator_table_id)
[print(item) for item in indicator_table.schema]
client.list_rows(indicator_table, max_results=5).to_dataframe()

SchemaField('country_name', 'STRING', 'NULLABLE', '', (), None)
SchemaField('country_code', 'STRING', 'NULLABLE', '', (), None)
SchemaField('indicator_name', 'STRING', 'NULLABLE', '', (), None)
SchemaField('indicator_code', 'STRING', 'NULLABLE', '', (), None)
SchemaField('value', 'FLOAT', 'NULLABLE', '', (), None)
SchemaField('year', 'INTEGER', 'NULLABLE', '', (), None)


/Users/blantj/opt/anaconda3/lib/python3.7/site-packages/google/cloud/bigquery/table.py:1690: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  if not self._validate_bqstorage(bqstorage_client, create_bqstorage_client):


,country_name,country_code,indicator_name,indicator_code,value,year
0,Syrian Arab Republic,SYR,Population in urban agglomerations of more tha...,EN.URB.MCTY.TL.ZS,2.819893e+01,1965
1,Syrian Arab Republic,SYR,"Population, male",SP.POP.TOTL.MA.IN,2.731856e+06,1965
2,Syrian Arab Republic,SYR,"Population, male",SP.POP.TOTL.MA.IN,3.011032e+06,1968
3,Syrian Arab Republic,SYR,"Population, male (% of total population)",SP.POP.TOTL.MA.ZS,5.057848e+01,1993
4,Syrian Arab Republic,SYR,"Portfolio equity, net inflows (BoP, current US$)",BX.PEF.TOTL.CD.WD,0.000000e+00,2003


Query country indicators from indicators_data table

In [13]:
indicators_query = """
        SELECT *
        FROM `bigquery-public-data.world_bank_wdi.indicators_data`
        WHERE indicator_name IN ('Population density (people per sq. km of land area)', 
                                 'GDP per capita (constant 2010 US$)',
                                 'GDP per capita growth (annual %)',
                                 'Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)',
                                 'Urban population (% of total population)',
                                 'Labor force with advanced education (% of total working-age population with advanced education)',
                                 'General government final consumption expenditure (% of GDP)',
                                 'Personal remittances, received (% of GDP)',
                                 'Prevalence of HIV, total (% of population ages 15-49)',
                                 'Self-employed, total (% of total employment) (modeled ILO estimate)')"""
indicators_query_job = client.query(indicators_query)
indicators_df = indicators_query_job.to_dataframe()
indicators_df.to_csv('Excel/indicators.csv', index=False)

/Users/blantj/opt/anaconda3/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Read indicators.csv file into pandas indicators df

In [14]:
indicators_df = pd.read_csv('Excel/indicators.csv')
print(indicators_df.shape)
indicators_df.head()

(92395, 6)


,country_name,country_code,indicator_name,indicator_code,value,year
0,East Asia & Pacific,EAS,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,5.739960,2007
1,East Asia & Pacific (IDA & IBRD countries),TEA,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,7.615462,1996
2,Euro area,EMU,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,5.436296,1962
3,Guyana,GUY,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,-13.136132,1982
4,Haiti,HTI,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,1.248083,1968


Filter indicators df to only include indicator values for most recently available year

In [15]:
def indicator_filter(row):
    years = indicators_df[(indicators_df['country_code'] == row['country_code']
                          )&(indicators_df['indicator_code'] == row['indicator_code'])]['year'].values
    if row['year'] == max(years):
        return 1
    else:
        return 0
indicators_df['filter'] = indicators_df.apply(indicator_filter, axis=1)
indicators_df_filtered = indicators_df[indicators_df['filter']==1]
print(indicators_df_filtered.shape)
indicators_df_filtered.head()

(2289, 7)


,country_name,country_code,indicator_name,indicator_code,value,year,filter
56,South Asia,SAS,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,3.611123,2019,1
60,Greenland,GRL,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,3.492548,2018,1
126,Heavily indebted poor countries (HIPC),HPC,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,1.278160,2019,1
157,"Venezuela, RB",VEN,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,-4.731050,2014,1
225,Equatorial Guinea,GNQ,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,-8.849975,2019,1


In [16]:
indicators_df_filtered.head(5)

,country_name,country_code,indicator_name,indicator_code,value,year,filter
56,South Asia,SAS,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,3.611123,2019,1
60,Greenland,GRL,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,3.492548,2018,1
126,Heavily indebted poor countries (HIPC),HPC,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,1.278160,2019,1
157,"Venezuela, RB",VEN,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,-4.731050,2014,1
225,Equatorial Guinea,GNQ,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,-8.849975,2019,1


Pivot indicators_df_filtered to include countries as rows and indicators as columns

In [17]:
pivoted_indicators_df = pd.pivot(indicators_df_filtered, index='country_name', columns='indicator_name', 
                                 values='value')
pivoted_indicators_df.reset_index(inplace=True)
print(pivoted_indicators_df.shape)
pivoted_indicators_df.head()

(263, 11)


indicator_name,country_name,GDP per capita (constant 2010 US$),GDP per capita growth (annual %),General government final consumption expenditure (% of GDP),"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)",Labor force with advanced education (% of total working-age population with advanced education),"Personal remittances, received (% of GDP)",Population density (people per sq. km of land area),"Prevalence of HIV, total (% of population ages 15-49)","Self-employed, total (% of total employment) (modeled ILO estimate)",Urban population (% of total population)
0,Afghanistan,571.466641,0.549628,13.123956,21.771000,64.676697,4.337765,56.937760,0.1,82.250000,25.754
1,Albania,5209.362832,2.650431,11.531084,46.603001,62.701599,9.640036,104.612263,0.1,54.459000,61.229
2,Algeria,4710.583772,-1.130692,18.602840,14.556000,NaN,1.054191,17.730075,0.1,31.554001,73.189
3,American Samoa,9271.712010,2.470435,NaN,NaN,NaN,NaN,277.325000,NaN,NaN,87.147
4,Andorra,45887.302126,1.669680,NaN,NaN,NaN,NaN,163.842553,NaN,NaN,87.984


Merge pivoted_indicators_df and fertility_df_filtered

In [18]:
merged_df = pd.merge(fertility_df_filtered, pivoted_indicators_df, how='inner', left_on='country_name', 
         right_on='country_name')
print(merged_df.shape)
merged_df.head()

(188, 15)


,country_code,country_name,year,total_fertility_rate,replacement,GDP per capita (constant 2010 US$),GDP per capita growth (annual %),General government final consumption expenditure (% of GDP),"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)",Labor force with advanced education (% of total working-age population with advanced education),"Personal remittances, received (% of GDP)",Population density (people per sq. km of land area),"Prevalence of HIV, total (% of population ages 15-49)","Self-employed, total (% of total employment) (modeled ILO estimate)",Urban population (% of total population)
0,AA,Aruba,2018,1.8324,0,26630.205281,1.521780,16.687991,NaN,NaN,1.836798,588.027778,NaN,NaN,43.546
1,AC,Antigua and Barbuda,2018,1.9906,0,15703.028476,3.753873,NaN,NaN,NaN,1.429945,218.831818,NaN,NaN,24.506
2,AE,United Arab Emirates,2018,1.7269,0,41420.483032,0.225275,13.278419,52.481998,77.567596,NaN,135.609110,NaN,3.902000,86.789
3,AF,Afghanistan,2018,5.0200,1,571.466641,0.549628,13.123956,21.771000,64.676697,4.337765,56.937760,0.1,82.250000,25.754
4,AG,Algeria,2018,2.6646,1,4710.583772,-1.130692,18.602840,14.556000,NaN,1.054191,17.730075,0.1,31.554001,73.189


Save final unscrubbed dataset to csv

In [19]:
merged_df.to_csv('Excel/unscrubbed_dataset.csv', index=False)

# Scrub Data

Read unscrubbed dataset into pandas and view df.info() overview of dataswt

In [20]:
unscrubbed_df = pd.read_csv('Excel/unscrubbed_dataset.csv')
unscrubbed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 15 columns):
 #   Column                                                                                           Non-Null Count  Dtype  
---  ------                                                                                           --------------  -----  
 0   country_code                                                                                     188 non-null    object 
 1   country_name                                                                                     188 non-null    object 
 2   year                                                                                             188 non-null    int64  
 3   total_fertility_rate                                                                             188 non-null    float64
 4   replacement                                                                                      188 non-null    int64  
 5   GDP per 

Drop columns with large numbers of missing values or descriptive values not used in modeling

In [21]:
unscrubbed_df.drop(columns=['Prevalence of HIV, total (% of population ages 15-49)',
                           'Labor force with advanced education (% of total working-age population with advanced education)',
                            'country_code', 'country_name', 'year']
                           , inplace=True)

Drop rows missing more than 2 indicators

In [22]:
unscrubbed_df = unscrubbed_df[unscrubbed_df.isnull().apply(lambda x: sum(x), axis=1)<=2]
print(unscrubbed_df.shape)
unscrubbed_df.head()

(169, 10)


,total_fertility_rate,replacement,GDP per capita (constant 2010 US$),GDP per capita growth (annual %),General government final consumption expenditure (% of GDP),"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)","Personal remittances, received (% of GDP)",Population density (people per sq. km of land area),"Self-employed, total (% of total employment) (modeled ILO estimate)",Urban population (% of total population)
0,1.8324,0,26630.205281,1.521780,16.687991,NaN,1.836798,588.027778,NaN,43.546
2,1.7269,0,41420.483032,0.225275,13.278419,52.481998,NaN,135.609110,3.902000,86.789
3,5.0200,1,571.466641,0.549628,13.123956,21.771000,4.337765,56.937760,82.250000,25.754
4,2.6646,1,4710.583772,-1.130692,18.602840,14.556000,1.054191,17.730075,31.554001,73.189
5,1.8881,0,5879.992654,1.366684,10.827243,63.130001,2.653967,120.234317,67.792000,56.031


Fill missing values with column mean

In [23]:
unscrubbed_df.fillna(value=unscrubbed_df.mean(), axis=0, inplace=True)
unscrubbed_df.head()

,total_fertility_rate,replacement,GDP per capita (constant 2010 US$),GDP per capita growth (annual %),General government final consumption expenditure (% of GDP),"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)","Personal remittances, received (% of GDP)",Population density (people per sq. km of land area),"Self-employed, total (% of total employment) (modeled ILO estimate)",Urban population (% of total population)
0,1.8324,0,26630.205281,1.521780,16.687991,52.654202,1.836798,588.027778,40.922914,43.546
2,1.7269,0,41420.483032,0.225275,13.278419,52.481998,4.847161,135.609110,3.902000,86.789
3,5.0200,1,571.466641,0.549628,13.123956,21.771000,4.337765,56.937760,82.250000,25.754
4,2.6646,1,4710.583772,-1.130692,18.602840,14.556000,1.054191,17.730075,31.554001,73.189
5,1.8881,0,5879.992654,1.366684,10.827243,63.130001,2.653967,120.234317,67.792000,56.031


View unscrubbed_df.info() overview of data after scrubbing

In [24]:
unscrubbed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169 entries, 0 to 186
Data columns (total 10 columns):
 #   Column                                                                                           Non-Null Count  Dtype  
---  ------                                                                                           --------------  -----  
 0   total_fertility_rate                                                                             169 non-null    float64
 1   replacement                                                                                      169 non-null    int64  
 2   GDP per capita (constant 2010 US$)                                                               169 non-null    float64
 3   GDP per capita growth (annual %)                                                                 169 non-null    float64
 4   General government final consumption expenditure (% of GDP)                                      169 non-null    float64
 5   Labor fo

Save scrubbed data to csv file

In [25]:
unscrubbed_df.to_csv('Excel/scrubbed_df.csv', index=False)